# FASTER RCNN (Prebuilt)

In [ ]:
# object_classes = ["LIKE", "DISLIKE", "STAR", "TSON", "AD", "ADLOADER"]

import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [ ]:
def get_model_instance(num_classes):
    # load a model instance pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # print(model)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# num_classes = 6

# model = get_model_instance(num_classes)
model.to(device)

In [ ]:

# model.load_state_dict(torch.load("C:\\Users\\91998\\runs\\detect\\train15\\weights\\best.pt"))
# checkpoint = torch.load('C:\\Users\\91998\\Downloads\\trained-visual-cue-detection-model.pth', map_location=torch.device('cpu'))

checkpoint = torch.load("C:\\Users\\91998\\runs\\detect\\train15\\weights\\best.pt", map_location=torch.device('cpu'))


In [ ]:
model.load_state_dict(checkpoint)

model.eval()

## Pipeline

In [ ]:
import time
import datetime
import pyautogui
from PIL import Image
import cv2
import numpy

In [ ]:
def pipeline(model,i):
    screenshot = pyautogui.screenshot()
    screenshot = screenshot.resize((640, 640))
    image = screenshot.convert("RGB")
    transform = transforms.Compose([transforms.ToTensor()])
    input_data = transform(image).unsqueeze(0) 
    output = model(input_data)
    print(output)

    # Get only >75%
    bounding_boxes = []
    for i, score in enumerate(output[0]['scores']):
        if score >= 0.75:
            bounding_boxes.append(i)

    # Make bounding boxes on all of them
    image = cv2.cvtColor(numpy.array(image), cv2.COLOR_RGB2BGR)
    for box in bounding_boxes:
        print(box)
        # Define the coordinates of the top-left corner
        # Define the width and height of the bounding box
        x, y, x2, y2 = map(int, output[0]['boxes'][box])
        print(2)
        # Calculate the coordinates of the other corners of the bounding box
        # x2, y2 = x + width, y + height

        # Draw the bounding box on the image
        color = (0, 255, 0)  # Green color, you can change it as needed
        thickness = 2  # Line thickness
        cv2.rectangle(image, (x, y), (x2, y2), color, thickness)
    print(3)
    # Display the image with the bounding box
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow('output', image)
    cv2.waitKey(0)
    cv2.imwrite(f'C:\\Users\\91998\\Downloads\\DarkPatterns\\downloaded_images\\test{i}.png', image)


In [ ]:
import time
for i in range(5):
    pipeline(model,i)
    time.sleep(5)

In [ ]:
# 1080 x 1920
screenshot = pyautogui.screenshot()
screenshot = screenshot.resize((1080, 1920))
screenshot.show()

In [ ]:
from PIL import Image

image_path = "C:\\Users\\91998\\Downloads\\AidUI-Object-Detection-Dataset-Master\\AidUI-Object-Detection-Dataset-Master\\10726_DISLIKE_2.png"
# image_path = "C:\\Users\\91998\\Downloads\\amazon.png"
# image_path = "C:\\Users\\91998\\Downloads\\AidUI-Evaluation-Dataset\\evaluation_dataset\\web\\images\\centralvapors.com.png"
image = Image.open(image_path).convert("RGB")

In [ ]:
# transform = transforms.Compose([
#     transforms.Resize((32, 32)),  # Adjust size if needed
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])
# transforms.Resize(224),
transform = transforms.Compose([transforms.ToTensor(),
                                      #transforms.Normalize([0.485, 0.456, 0.406],
                                      #                     [0.229, 0.224, 0.225])
                                     ])

input_data = transform(image).unsqueeze(0) 

In [ ]:

image=image.resize((64,64))
image.show()

In [ ]:
output = model(input_data)


In [ ]:
output

In [ ]:
output[0]['scores'][0] > 0.5

In [ ]:
output[0]['boxes'][0]
output[0]['scores']


In [ ]:
import cv2

# Load the image
image = cv2.imread(image_path)

print(1)
# Define the coordinates of the top-left corner
# Define the width and height of the bounding box
x, y, x2, y2 = map(int, output[0]['boxes'][0])
print(2)
# Calculate the coordinates of the other corners of the bounding box
# x2, y2 = x + width, y + height

# Draw the bounding box on the image
color = (0, 255, 0)  # Green color, you can change it as needed
thickness = 2  # Line thickness
cv2.rectangle(image, (x, y), (x2, y2), color, thickness)
print(3)
# Display the image with the bounding box
cv2.namedWindow("output", cv2.WINDOW_NORMAL)
cv2.imshow('output', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
image = cv2.imread(image_path)
cv2.namedWindow("output", cv2.WINDOW_NORMAL)
cv2.imshow('output', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# YOLO V8

https://stackoverflow.com/questions/4081064/how-to-get-the-number-of-pixels-a-user-has-scrolled-down-the-page

In [ ]:
device = torch.device("cpu")
model.todevice()

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data='C:/Users/91998/Downloads/DarkPatterns/dataset.yml', epochs=10, imgsz=1280, device=[0], patience=5)
# "C:\Users\91998\Downloads\DarkPatterns\output\dataset.yml"

In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from YAML
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
# model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

# Train the model
results = model.train(data='C:/Users/91998/Downloads/DarkPatterns/dataset.yml', epochs=10, imgsz=1280)

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import torchvision
print(torchvision.__version__)

In [2]:
from PIL import Image
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
# model = YOLO('C:\\Users\\91998\\Downloads\\DarkPatterns\\yolov8n.pt')
model = YOLO("C:\\Users\\91998\\runs\\detect\\train15\\weights\\best.pt")


In [41]:
# Run inference on 'bus.jpg'
# results = model("C:\\Users\\91998\\Downloads\\checkbox.png")  # results list
im_path = "C:\\Users\\91998\\Downloads\\DarkPatterns\\output\\images\\train\\images\\(688)www.chicwish.com_4cb6_BOMB_3.png"
image = cv2.imread(im_path)
results = model(im_path)
# Show the results
for r in results:
    for i, prob in enumerate(r.boxes.conf):
        if prob > 0.5:
            # im_array = r.plot()  # plot a BGR numpy array of predictions
            # im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
            # im.show()  # show image
            # im.save('results.jpg')  # save image

            color = (0, 255, 0)  # Green color, you can change it as needed
            thickness = 2  # Line thickness
            x, y, x2, y2 = map(int, r.boxes.xywh[i])
            cv2.rectangle(image, (x, y), (x2, y2), color, thickness)
            # print(3)
            # Display the image with the bounding box
    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow('output', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

image 1/1 C:\Users\91998\Downloads\DarkPatterns\output\images\train\images\(688)www.chicwish.com_4cb6_BOMB_3.png: 704x1280 1 BOMB, 271.2ms
Speed: 14.9ms preprocess, 271.2ms inference, 1.0ms postprocess per image at shape (1, 3, 704, 1280)


In [42]:
# import numpy as np
results[0].boxes
# results[0].probs

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([2.])
conf: tensor([0.9534])
data: tensor([[1.6752e+03, 8.7411e+02, 1.7023e+03, 9.1864e+02, 9.5344e-01, 2.0000e+00]])
id: None
is_track: False
orig_shape: (1300, 2400)
shape: torch.Size([1, 6])
xywh: tensor([[1688.7686,  896.3747,   27.0760,   44.5347]])
xywhn: tensor([[0.7037, 0.6895, 0.0113, 0.0343]])
xyxy: tensor([[1675.2305,  874.1074, 1702.3065,  918.6420]])
xyxyn: tensor([[0.6980, 0.6724, 0.7093, 0.7066]])

In [17]:
for r in results:
    boxes = r.boxes.numpy()  # Assuming Tensor conversion
    probs = r.probs.numpy()  # Assuming Tensor conversion

    for box in boxes:
        x1, y1, x2, y2 = box  # Extract box coordinates (replace with correct indices)
        class_id = np.argmax(probs)  # Assuming first index is confidence, others are class probs
        class_prob = probs[class_id]  # Access probability for detected class

        print(f"Bounding box: ({x1}, {y1}) - ({x2}, {y2}), Class {class_id} with probability {class_prob}")

AttributeError: 'NoneType' object has no attribute 'numpy'

In [1]:
import time
import datetime
import pyautogui
from PIL import Image
import cv2
import numpy

In [8]:
def pipeline(model,i):
    screenshot = pyautogui.screenshot()
    image = screenshot.convert("RGB")
    
    results = model(image)
    # print(output)

    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.show()  # show image
        im.save(f'C:\\Users\\91998\\Downloads\\DarkPatterns\\downloaded_images\\test{i}.png')
    print(3)
    # Display the image with the bounding box
    # cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    # cv2.imshow('output', image)
    # cv2.waitKey(0)
    # cv2.imwrite(f'C:\\Users\\91998\\Downloads\\DarkPatterns\\downloaded_images\\test{i}.png', image)


In [10]:
import time
for i in range(5):
    pipeline(model,i)
    time.sleep(5)

0: 800x1280 (no detections), 382.2ms
Speed: 14.3ms preprocess, 382.2ms inference, 1.1ms postprocess per image at shape (1, 3, 800, 1280)
3

0: 800x1280 1 CHECKBOX, 242.2ms
Speed: 9.1ms preprocess, 242.2ms inference, 2.0ms postprocess per image at shape (1, 3, 800, 1280)
3

0: 800x1280 1 BOMB, 244.8ms
Speed: 11.3ms preprocess, 244.8ms inference, 1.2ms postprocess per image at shape (1, 3, 800, 1280)
3

0: 800x1280 (no detections), 260.0ms
Speed: 12.7ms preprocess, 260.0ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 1280)
3

0: 800x1280 (no detections), 262.1ms
Speed: 10.3ms preprocess, 262.1ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 1280)
3
